Козубенко, ДЗ к уроку 3

Собрать информацию о вакансиях на вводимую должность с сайтов hh.ru и/или Superjob и/или работа.ру. Приложение должно анализировать несколько страниц сайта. Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (дополнительно: разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия

In [252]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import pprint
from time import sleep

In [253]:
def salary_clean(salary_text):
  min_sal = None
  max_sal = None
  curr_sal = None
  salary_text_list = salary_text.replace('\u202f', '').split()
  for i in range(len(salary_text_list)):
    if salary_text_list[i] == 'от':
      min_sal = int(salary_text_list[i + 1])
    elif salary_text_list[i] == 'до':
      max_sal = int(salary_text_list[i + 1])
    elif salary_text_list[i] == '–':
      min_sal = int(salary_text_list[i - 1])
      max_sal = int(salary_text_list[i + 1])
  curr_sal = salary_text_list[-1]
  return [min_sal, max_sal, curr_sal]

In [254]:
url = 'https://vladivostok.hh.ru/search/vacancy'
headers = {
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
}
params = {
    'text': 'Водитель',
    'from': 'suggest_post',
    'salary': '',
    'clusters': 'true',
    'area': '22',
    'no_magic': 'true',
    'ored_clusters': 'true',
    'enable_snippets': 'true'
}

In [255]:
# Определяю кол-во страниц для парсинга
response = requests.get(url=url, headers=headers, params=params)
soup = bs(response.text, 'html.parser')
index_last_page_t = soup.find('a', {'data-qa': 'pager-next'}).previous_sibling.text
index_last_page = int(index_last_page_t) - 1
index_last_page

5

In [256]:
url = 'https://vladivostok.hh.ru/search/vacancy'
headers = {
    'User-Agent':	'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
}
page_index = ''
params = {
    'text': 'Водитель',
    'from': 'suggest_post',
    'salary': '',
    'clusters': 'true',
    'area': '22',
    'no_magic': 'true',
    'ored_clusters': 'true',
    'enable_snippets': 'true',
    'page': page_index,
    'hhtmFrom': 'vacancy_search_list'
}

In [257]:
response_text = ''

In [258]:
# index_last_page = 2

In [259]:
i = 0
while i <= index_last_page:
  sleep(0.5)
  page_index = str(i)
  # print(page_index)
  params = {
    'text': 'Водитель',
    'from': 'suggest_post',
    'salary': '',
    'clusters': 'true',
    'area': '22',
    'no_magic': 'true',
    'ored_clusters': 'true',
    'enable_snippets': 'true',
    'page': page_index,
    'hhtmFrom': 'vacancy_search_list'
}
  # print(params['page'])
  response = requests.get(url=url, headers=headers, params=params)
  if response.status_code != 200:
    print('Parsing ended')
    break
  else: 
    response_text += response.text
    i += 1

In [260]:
soup = bs(response_text, 'html.parser') 

In [261]:
# soup

In [262]:
jobs = soup.find_all('a', {'class': 'serp-item__title'})
len(jobs)

114

In [263]:
salary_list = []
employee_list = []
salary_data = soup.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
for s in salary_data:
  sal = s.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
  # print(sal.text)
  try:
    salary_list.append(sal.text)
  except AttributeError:
    salary_list.append(None)
  employee = s.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
  employee_list.append(employee.text.replace('\xa0', ' '))
# salary_list
# employee_list

In [264]:
jobs_data = {
    'job_title': [],
    'employee': [],
    'salary_min': [],
    'salary_max': [],
    'salary_currency': [],
    'job_link': [],
    'source': []
}

In [265]:
# salary_clean(salary_list[1])[0]

In [266]:
i = 0
for job in jobs:
  jobs_data['job_title'].append(job.text)
  jobs_data['job_link'].append(job['href'])
  jobs_data['source'].append(url)
  try:
    jobs_data['salary_min'].append(salary_clean(salary_list[i])[0])
  except AttributeError:
    jobs_data['salary_min'].append(None)
  try:  
    jobs_data['salary_max'].append(salary_clean(salary_list[i])[1])
  except AttributeError:
    jobs_data['salary_max'].append(None)
  try:
    jobs_data['salary_currency'].append(salary_clean(salary_list[i])[2])
  except AttributeError:
    jobs_data['salary_currency'].append(None)
  jobs_data['employee'].append(employee_list[i])
  i += 1
for title in  jobs_data:
  print(title, len(jobs_data[title]))

job_title 114
employee 114
salary_min 114
salary_max 114
salary_currency 114
job_link 114
source 114


In [267]:
jobs_df = pd.DataFrame(data=jobs_data)
jobs_df

,job_title,employee,salary_min,salary_max,salary_currency,job_link,source
0,"Водитель на седельный тягач С, Е",ООО ПМТК,100000.0,NaN,руб.,https://vladivostok.hh.ru/vacancy/68771078?fro...,https://vladivostok.hh.ru/search/vacancy
1,Водитель с личным автомобилем,i'way,150000.0,NaN,руб.,https://vladivostok.hh.ru/vacancy/69693913?fro...,https://vladivostok.hh.ru/search/vacancy
2,Водитель на служебный автомобиль,ООО ЛОТТЕ Отель Владивосток,50000.0,50000.0,руб.,https://vladivostok.hh.ru/vacancy/69830143?fro...,https://vladivostok.hh.ru/search/vacancy
3,Водитель-экспедитор,ООО ОВИТА.ру,90000.0,NaN,руб.,https://vladivostok.hh.ru/vacancy/69792248?fro...,https://vladivostok.hh.ru/search/vacancy
4,Водитель категории В,Глобал HR (ИП Вяткин Максим Анатольевич),65000.0,NaN,руб.,https://vladivostok.hh.ru/vacancy/69839478?fro...,https://vladivostok.hh.ru/search/vacancy
...,...,...,...,...,...,...,...
109,Оператор дробильных установок,ООО Океанстрой,120000.0,NaN,руб.,https://vladivostok.hh.ru/vacancy/68827099?fro...,https://vladivostok.hh.ru/search/vacancy
110,Водитель погрузчика,Группа компаний Монарх,50000.0,NaN,руб.,https://vladivostok.hh.ru/vacancy/68902976?fro...,https://vladivostok.hh.ru/search/vacancy
111,Водитель-курьер в ресторан (подработка),ООО Департамент Ф53,45000.0,NaN,руб.,https://vladivostok.hh.ru/vacancy/68869626?fro...,https://vladivostok.hh.ru/search/vacancy
112,Водитель-курьер на личном автомобиле (Delivery...,ProfiMonsters,65000.0,95000.0,руб.,https://vladivostok.hh.ru/vacancy/68861320?fro...,https://vladivostok.hh.ru/search/vacancy


In [268]:
jobs_df.to_csv('output.csv')